In [1]:
from Unet import build_model

import torch
import os
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import numpy as np
import base64
import sys
import os

torch.cuda.empty_cache()
def ce_loss(y_true, y_pred):
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    term_1 = y_true * K.log(y_pred + K.epsilon())
    out = -K.mean(term_0 + term_1)
    return out

def acc_met(y_true,y_pred):
    out = K.mean(K.equal(y_true, y_pred))
    return out

def sensitivity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_pred = 1 - y_pred_f
    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f * neg_y_pred)
    return tp / (tp+fn+K.epsilon())

def specificity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_true = 1 - y_true_f
    neg_y_pred = 1 - y_pred_f
    fp = K.sum(neg_y_true * y_pred_f)
    tn = K.sum(neg_y_true * neg_y_pred)
    return tn / (tn + fp + K.epsilon())


# Give Image in RGB Format
def run(img_path):
    # print(image_path)
    # img_path = sys.argv[1]
    # print(img_path)
    
    img = cv2.imread(img_path)
    # print(img)
    # gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # print(gray_img)
    # print(img)
    # Encode the processed image as base64
    # _, buffer = cv2.imencode('.jpg', gray_img)
    # encoded_image = base64.b64encode(buffer).decode('utf-8')
    
    # print(encoded_image)  # Print the base64 encoded image data

    clahe = cv2.createCLAHE(clipLimit = 8, tileGridSize=(8,8))
    if img.shape[0] == 3:
        img = img[1,:,:]
    elif img.shape[2] == 3:
        img = img[:,:,1]
    
    m,n = img.shape
    img = clahe.apply(img)
    
    if os.listdir("./Model")[0][-1] == 't':
        # print(img.shape)
        final_res = torch.zeros(img.shape)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        device = torch.device(device)
        model = build_model().to(device)
        model_filename = "./Model/Model.pt"
        state = torch.load(model_filename,map_location=torch.device(device))
        model.load_state_dict(state['model_state_dict'])
        model.eval()
        batch = []
        
        for i in range(0,m,8):
            for j in range(0,n,8):
                if(i+48 > m) or (j+48 > n):
                    continue

                cimg = img[i:i+48,j:j+48]

                batch.append(cimg)
                
        batch = torch.Tensor(batch).unsqueeze(1)
        with torch.no_grad():
            out = model(batch.to(device))
        
        # print(out.shape)
        oc = 0
        for i in range(0,m,8):
            for j in range(0,n,8):
                if(i+48 > m) or (j+48 > n):
                    continue
                final_res[i:i+48,j:j+48] = final_res[i:i+48,j:j+48] + out[oc][0].cpu().detach()
                oc+=1
            
        if final_res is None:
            raise ValueError("final_res is not initialized. Model may not have produced any output.")
        else:
            final_res = (final_res*255).numpy().astype(np.uint8)
            _, buffer = cv2.imencode('.jpg', final_res)
            encoded_image = base64.b64encode(buffer).decode('utf-8')
            if encoded_image is None:
                raise ValueError("final_res is not initialized. Model may not have produced any output.")
            else:
                print("1 image encoded: ", encoded_image)  # Print the base64 encoded image data
        
        # return final_res.numpy()
    
    else:
        seg_model = load_model("./UNet_JND_EOphtha.h5", custom_objects={"ce_loss":ce_loss, "acc_met":acc_met, "sensitivity": sensitivity, "specificity": specificity})
        final_res = np.zeros(img.shape)
        batch = []
        for i in range(0,m,8):
            for j in range(0,n,8):
                if(i+48 > m) or (j+48 > n):
                    continue

                cimg = img[i:i+48,j:j+48]
                batch.append(cimg)
        
        batch = np.array(batch)
        batch = np.expand_dims(batch,axis=-1)
        
        with torch.no_grad():
            out = seg_model.predict(batch,verbose=False)
        for i in range(0,out.shape[0]):
            final_res[i:i+48,j:j+48] = final_res[i:i+48,j:j+48] + out[0,:,:,0]

        if final_res is None:
            raise ValueError("final_res is not initialized. Model may not have produced any output.")
        else:
            final_res = final_res.astype(np.uint8)
            _, buffer = cv2.imencode('.jpg', final_res)
            encoded_image = base64.b64encode(buffer).decode('utf-8')
            if encoded_image is None:
                raise ValueError("final_res is not initialized. Model may not have produced any output.")
            else:
                print("2 image encoded: ", encoded_image)  # Print the base64 encoded image data
        
        # return final_res
    
if __name__ == "__main__":
    # Get the current directory of the Python script
    current_directory = os.path.dirname(os.path.abspath('run.py'))

    # Navigate one level up
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

    # Construct the path to the image file
    image_path = os.path.join(parent_directory, 'epotha.jpg')
    run(image_path)

1 image encoded:  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/wAALCAAwADABAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APwEu0skupUsLiWSBXIgkmhCO654LKGYKcckAnB4yetNgWB5QtzKyIfvMibiPwyP50hC4GH7c/nTT7U6NSzZBUbRnLEf16/SpLpJI3aI3KOF67DhevYcdye3rUe07QzH5SxHB+maRWZc7T1GKt3OpapJpEWjXHFtBM00amMAh3C5YnGSSFXr2UYqt5ciRLcBgAzEDDjORg9Oo6jnvz6GpGuL2Qq73DbrZRsZ5MMoBAAXJzwew6cn1qS7tdTureTxBeurCaf53knXzJGYud23O4jKtlgMA8E5IzBCszI+y3MiqhZiAfkGQN3HTnA545okWfCyTI4XO1SQccY4GfQEfmKDbSIFeRSFboy4PYE/oaj9qfmFgoKFfnO5gc8ccAe3Perl+9nNbWsVnA6lLY+b5ZZhJJubLnd907QuQBj5fxqidp5HHsaCSTkgfgKsQX3ljAgQkxhTlnAPOdxIYcjAHpUU6KjbVdW5OSvTr/8AW/WozkHBFX7yPVpNKg1C706VrUKbaC8ZG2bgfMKhuhYBxkdgw471Ugie5I

C:\Users\DELL\AppData\Local\Temp\ipykernel_3016\2497224119.py:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  batch = torch.Tensor(batch).unsqueeze(1)


In [2]:
import torch

In [32]:
type(out)

numpy.ndarray

In [37]:
t = cv2.imread('../epotha.jpg',0)

In [40]:
t.dtype

dtype('uint8')

In [42]:
out = cv2.GaussianBlur(t,(5,5),0)
out = cv2.threshold(out,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [44]:
out = cv2.GaussianBlur(t,(5,5),cv2.BORDER_DEFAULT)
_,out = cv2.threshold(out,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [47]:
out.shape

(48, 48)

In [14]:
def ce_loss(y_true, y_pred):
    term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    term_1 = y_true * K.log(y_pred + K.epsilon())
    out = -K.mean(term_0 + term_1)
    return out

def acc_met(y_true,y_pred):
    out = K.mean(K.equal(y_true, y_pred))
    return out

def sensitivity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_pred = 1 - y_pred_f
    tp = K.sum(y_true_f * y_pred_f)
    fn = K.sum(y_true_f * neg_y_pred)
    return tp / (tp+fn+K.epsilon())

def specificity(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    neg_y_true = 1 - y_true_f
    neg_y_pred = 1 - y_pred_f
    fp = K.sum(neg_y_true * y_pred_f)
    tn = K.sum(neg_y_true * neg_y_pred)
    return tn / (tn + fp + K.epsilon())

In [19]:
from tensorflow.keras.models import load_model
seg_model = load_model("./Model/Model.h5", custom_objects={"ce_loss":ce_loss, "acc_met":acc_met, "sensitivity": sensitivity, "specificity": specificity})

TypeError: string indices must be integers, not 'str'